# Probing part 4

Vantage points will probe the targets in a 3-step method, either by doing pings or traceroutes.  

Vantage points are the Ripe Atlas anchors, then indireclty some online landmarks.  
As always, targets are the anchors.  

This notebook is an implementation of the street level method. Check the paper for more information.  
To do after create_datasets.ipynb

In [2]:
import traceback

from pprint import pprint
from clickhouse_driver import Client

from scripts.utils.file_utils import load_json, dump_json
from scripts.utils.helpers import haversine
from scripts.street_level.traceroutes_results import serialize
from scripts.street_level.three_tiers import get_all_info_geoloc
from default import USER_ANCHORS_FILE, ANALYZABLE_FILE, CLICKHOUSE_DB

### database for traceroutes

In [4]:
# TODO: why create this db? no insert data

# Create Database
client = Client(CLICKHOUSE_DB)
query = 'CREATE DATABASE street_lvl'
client.execute(query)

# Create Traceroutes table
query = (
        f"create table street_lvl.street_lvl_traceroutes"
        f"(src_addr String, dst_prefix String, dst_addr String, resp_addr String, proto Int16, hop Int16, rtt Float64, ttl Int16, prb_id Int64, msm_id Int64, tstamp Datetime('UTC'))"
        f"ENGINE = MergeTree()"
        f"order by (dst_addr, src_addr, tstamp)"
)
client.execute(query)
client.disconnect()

### Main

This would take a lot of time (more than a day)

In [2]:
# Anchors are the targets and Vantage points
anchors = load_json(USER_ANCHORS_FILE)
try:
    all_res = load_json(ANALYZABLE_FILE)
except FileNotFoundError:
    all_res = {}

i = 0
for target in anchors:
    try:
        target_ip = target['address_v4']
        if target_ip in all_res: # we skip targets already geolocated
            continue
        print(f"{i}:{target_ip}")
        i += 1

        res = get_all_info_geoloc(target_ip)
        res = serialize(res)
        # We save the coordinates of the targets as given by RIPE Atlas
        res['RIPE:lat'] = target['geometry']['coordinates'][1]
        res['RIPE:lon'] = target['geometry']['coordinates'][0]

        # We save the error of the estimated geolocation at each step
        if res['lat'] != None and res['lon'] != None:
            res['error'] = haversine(
                (res['lat'], res['lon']), (res['RIPE:lat'], res['RIPE:lon']))
        if 'tier1:lat' in res and 'tier1:lon' in res and res['tier1:lat'] != None and res['tier1:lon'] != None:
            res['tier1:error'] = haversine(
                (res['tier1:lat'], res['tier1:lon']), (res['RIPE:lat'], res['RIPE:lon']))
        if 'tier2:lat' in res and 'tier2:lon' in res and res['tier2:lat'] != None and res['tier2:lon'] != None:
            res['tier2:error'] = haversine(
                (res['tier2:lat'], res['tier2:lon']), (res['RIPE:lat'], res['RIPE:lon']))
        if 'tier3:lat' in res and 'tier3:lon' in res and res['tier3:lat'] != None and res['tier3:lon'] != None:
            res['tier3:error'] = haversine(
                (res['tier3:lat'], res['tier3:lon']), (res['RIPE:lat'], res['RIPE:lon']))

        all_res[target_ip] = res
        # We save the results
        dump_json(all_res, ANALYZABLE_FILE)
    except Exception:
        traceback.print_exc()

### Geolocat one IP

In [6]:
target_ip = '195.83.132.129' # LAAS/CNRS
geolocation = get_all_info_geoloc(target_ip)
#geolocation = geoloc(target_ip)
print(geolocation)
geolocation = serialize(geolocation)
dump_json(geolocation, 'res_tmp.json')

{'target_ip': '195.83.132.129', 'tier1:done': False, 'tier2:done': False, 'tier3:done': False, 'negative_rtt_included': True, 'speed_threshold': 0.6666666666666666, 'tier1:lat': None, 'tier1:lon': None, 'vps': set(), 'tier1:duration': 1282.0457310676575, 'lat': None, 'lon': None}
